Based on [Tensorflow tutorial](https://www.tensorflow.org/tutorials/text/word2vec?hl=en#prepare_training_data_for_word2vec).

To adapt w2v to book recomendation the books replace the words. List of rated books by an user is equivalent to a "sentence". The book vectorization is already made by the database.

Current issues :

*   try to fit the book data in this implementation ==> only keep a fix amount of book for each other (10 books)
  * need to make more lists for each user in order to have all the book represented??
Since every list is independent, need to make book appears several time for different list of each user with different books.
  * Increase the size of each list to fit the longest user list and fill the smaller ones with 0?
*   Not sure what the *vocab_size* is supposed to be, the number of unique books in the data set is slightly too small.
* Only use one dataset, may switch to others 


In [ ]:
import io
import itertools # This module implements a number of iterator building blocks inspired by constructs from APL, Haskell (brrr), and SML
import os
import re #regex
import string
import tensorflow as tf
import tqdm # make loops show a smart progress meter: 76%|████████████████████████       | 7568/10000 [00:33<00:10, 229.00it/s]
import pandas as pd
pd.set_option('display.max_colwidth', None)


from tensorflow.keras import Model, Sequential
from tensorflow.keras.layers import Activation, Dense, Dot, Embedding, Flatten, GlobalAveragePooling1D, Reshape
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
AUTOTUNE = tf.data.AUTOTUNE
SEED = 42 
# num_ns = 4


In [ ]:
rated = pd.read_csv("https://raw.githubusercontent.com/ArielNATAF/Books_project/dev/datasets/ratings.csv", usecols = ["user_id","book_id"])
rated = rated.sort_values('user_id')

rated["book_id"].nunique()
books = pd.read_csv("https://raw.githubusercontent.com/ArielNATAF/Books_project/dev/datasets/books.csv", usecols =['book_id', 'original_title'])
#rated = rated.set_index('book_id').join(books.set_index('book_id'))
rated.head()


,user_id,book_id
0,1,258
999485,1,140
999486,1,869
999487,1,2679
999488,1,1310


In [96]:
user_ratedbooks_ds = rated.groupby('user_id')['book_id'].apply(list).reset_index(name='book_list')
user_ratedbooks_ds = user_ratedbooks_ds.drop('user_id', 1)
user_ratedbooks_ds['book_list'].to_numpy()

user_ratedbooks_ds['book_list'] = user_ratedbooks_ds['book_list'].apply(lambda x: np.array(x))
user_ratedbooks_ds['book_list'] = user_ratedbooks_ds['book_list'].apply(lambda x: x[[0,1,2,3,4,5,6,7,8,9]])
# Bad way to only have 10 book for each list. Cut the "excess"
  # ==> Need to improve making of lists
# Since this method work I susect there is no list with less than 10 items.
  # ==> Need to implement a check of min size
print(user_ratedbooks_ds.head())
print(type(user_ratedbooks_ds['book_list'][0]))

                                                   book_list
0      [258, 140, 869, 2679, 1310, 414, 2738, 47, 867, 1796]
1               [131, 125, 630, 932, 8, 5, 264, 54, 66, 284]
2  [427, 5395, 3311, 4057, 1714, 118, 1005, 1928, 7401, 484]
3           [288, 495, 476, 325, 113, 45, 297, 11, 245, 301]
4  [1795, 305, 6014, 887, 326, 3037, 1742, 1339, 2592, 1145]
<class 'numpy.ndarray'>


In [ ]:
# Generates skip-gram pairs with negative sampling for a list of sequences
# (int-encoded sentences) based on window size, number of negative samples
# and vocabulary size.
def generate_training_data(sequences, window_size, num_ns, vocab_size, seed):
  # Elements of each training example are appended to these lists.
  targets, contexts, labels = [], [], []

  # Build the sampling table for vocab_size tokens.
  sampling_table = tf.keras.preprocessing.sequence.make_sampling_table(vocab_size)

  # Iterate over all sequences (sentences) in dataset.
  for sequence in tqdm.tqdm(sequences):

    # Generate positive skip-gram pairs for a sequence (sentence).
    positive_skip_grams, _ = tf.keras.preprocessing.sequence.skipgrams(
          sequence, 
          vocabulary_size=vocab_size,
          sampling_table=sampling_table,
          window_size=window_size,
          negative_samples=0)

    # Iterate over each positive skip-gram pair to produce training examples 
    # with positive context word and negative samples.
    for target_word, context_word in positive_skip_grams:
      context_class = tf.expand_dims(
          tf.constant([context_word], dtype="int64"), 1)
      negative_sampling_candidates, _, _ = tf.random.log_uniform_candidate_sampler(
          true_classes=context_class,
          num_true=1, 
          num_sampled=num_ns, 
          unique=True, 
          range_max=vocab_size, 
          seed=SEED, 
          name="negative_sampling")

      # Build context and label vectors (for one target word)
      negative_sampling_candidates = tf.expand_dims(
          negative_sampling_candidates, 1)

      context = tf.concat([context_class, negative_sampling_candidates], 0)
      label = tf.constant([1] + [0]*num_ns, dtype="int64")

      # Append each element from the training example to global lists.
      targets.append(target_word)
      contexts.append(context)
      labels.append(label)

  return targets, contexts, labels

In [97]:
# I am not sure what the vocab size is supposed to be,
# the number of unique book is slightly too small here
vocab_size = int(rated["book_id"].nunique()*1.2) 
vocab      = books["original_title"].tolist()
type(books["original_title"].tolist())

list

In [88]:
targets, contexts, labels = generate_training_data(
    sequences   = user_ratedbooks_ds['book_list'], 
    window_size = 2, 
    num_ns      = 4, 
    vocab_size  = vocab_size, 
    seed        = SEED)
print(len(targets), len(contexts), len(labels))

100%|██████████| 53424/53424 [01:01<00:00, 871.36it/s]


613461 613461 613461


In [ ]:
BATCH_SIZE = 1024
BUFFER_SIZE = 10000
dataset = tf.data.Dataset.from_tensor_slices(((targets, contexts), labels))
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)
print(dataset)
dataset = dataset.cache().prefetch(buffer_size=AUTOTUNE)
print(dataset)


<BatchDataset shapes: (((1024,), (1024, 5, 1)), (1024, 5)), types: ((tf.int32, tf.int64), tf.int64)>
<PrefetchDataset shapes: (((1024,), (1024, 5, 1)), (1024, 5)), types: ((tf.int32, tf.int64), tf.int64)>


In [ ]:
class Word2Vec(Model):
  def __init__(self, vocab_size, embedding_dim):
    super(Word2Vec, self).__init__()
    self.target_embedding = Embedding(vocab_size, 
                                      embedding_dim,
                                      input_length=1,
                                      name="w2v_embedding", )
    self.context_embedding = Embedding(vocab_size, 
                                       embedding_dim, 
                                       input_length=num_ns+1)
    self.dots = Dot(axes=(3,2))
    self.flatten = Flatten()

  def call(self, pair):
    target, context = pair
    we = self.target_embedding(target)
    ce = self.context_embedding(context)
    dots = self.dots([ce, we])
    return self.flatten(dots)


In [89]:
def custom_loss(x_logit, y_true):
   return tf.nn.sigmoid_cross_entropy_with_logits(logits=x_logit, labels=y_true)


embedding_dim = 128
word2vec = Word2Vec(vocab_size, embedding_dim)
word2vec.compile(optimizer='adam',
              loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir="logs")
word2vec.fit(dataset, epochs=15, callbacks=[tensorboard_callback])


Epoch 1/15
596/596 [==============================] - 23s 37ms/step - loss: 1.5320 - accuracy: 0.3016
Epoch 2/15
596/596 [==============================] - 22s 37ms/step - loss: 1.2694 - accuracy: 0.4334
Epoch 3/15
596/596 [==============================] - 22s 37ms/step - loss: 1.1713 - accuracy: 0.5152
Epoch 4/15
596/596 [==============================] - 22s 36ms/step - loss: 1.0234 - accuracy: 0.6212
Epoch 5/15
596/596 [==============================] - 22s 37ms/step - loss: 0.8853 - accuracy: 0.6918
Epoch 6/15
596/596 [==============================] - 22s 37ms/step - loss: 0.7633 - accuracy: 0.7475
Epoch 7/15
596/596 [==============================] - 22s 37ms/step - loss: 0.6527 - accuracy: 0.7982
Epoch 8/15
596/596 [==============================] - 21s 36ms/step - loss: 0.5536 - accuracy: 0.8424
Epoch 9/15
596/596 [==============================] - 21s 36ms/step - loss: 0.4673 - accuracy: 0.8780
Epoch 10/15
596/596 [==============================] - 22s 37ms/step - loss: 0.393

In [ ]:
%load_ext tensorboard

fashion_mnist = tf.keras.datasets.fashion_mnist

(x_train, y_train),(x_test, y_test) = fashion_mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

%tensorboard --logdir logs

In [ ]:
weights = word2vec.get_layer('w2v_embedding').get_weights()[0]
vocab = books["original_title"].tolist()

out_v = io.open('vectors.tsv', 'w', encoding='utf-8')
out_m = io.open('metadata.tsv', 'w', encoding='utf-8')

for index, word in enumerate(vocab):
  if  index == 0: continue # skip 0, it's padding.
  vec = weights[index] 
  out_v.write('\t'.join([str(x) for x in vec]) + "\n")
  out_m.write(str(word) + "\n")
out_v.close()
out_m.close()

In [ ]:
try:
  from google.colab import files
  files.download('vectors.tsv')
  files.download('metadata.tsv')
except Exception as e:
  pass

<

< `vectors.tsv` & `metadata.tsv` in the notebook repo

< 


---
text example

---



In [ ]:
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')


1122304/1115394 [==============================] - 0s 0us/step


In [ ]:
with open(path_to_file) as f: 
  lines = f.read().splitlines()
for line in lines[:20]:
  print(line)

text_ds = tf.data.TextLineDataset(path_to_file).filter(lambda x: tf.cast(tf.strings.length(x), bool))

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.


In [ ]:
# We create a custom standardization function to lowercase the text and 
# remove punctuation.
def custom_standardization(input_data):
  lowercase = tf.strings.lower(input_data)
  return tf.strings.regex_replace(lowercase,
                                  '[%s]' % re.escape(string.punctuation), '')

# Define the vocabulary size and number of words in a sequence.
vocab_size = 4096
sequence_length = 10

# Use the text vectorization layer to normalize, split, and map strings to
# integers. Set output_sequence_length length to pad all samples to same length.
vectorize_layer = TextVectorization(
    standardize=custom_standardization,
    max_tokens=vocab_size,
    output_mode='int',
    output_sequence_length=sequence_length)


In [ ]:
vectorize_layer.adapt(text_ds.batch(1024))
type(vectorize_layer)

tensorflow.python.keras.layers.preprocessing.text_vectorization.TextVectorization

In [ ]:
# Save the created vocabulary for reference.
inverse_vocab = vectorize_layer.get_vocabulary()
print(inverse_vocab[:20])


['', '[UNK]', 'the', 'and', 'to', 'i', 'of', 'you', 'my', 'a', 'that', 'in', 'is', 'not', 'for', 'with', 'me', 'it', 'be', 'your']


In [ ]:
def vectorize_text(text):
  text = tf.expand_dims(text, -1)
  return tf.squeeze(vectorize_layer(text))

# Vectorize the data in text_ds.
text_vector_ds = text_ds.batch(1024).prefetch(AUTOTUNE).map(vectorize_layer).unbatch()
text_vector_ds

<_UnbatchDataset shapes: (10,), types: tf.int64>

In [ ]:
sequences = list(text_vector_ds.as_numpy_iterator())
print(len(sequences))
for seq in sequences[:5]:
  print(f"{seq} => {[inverse_vocab[i] for i in seq]}")
  print(type (seq))


32777
[ 89 270   0   0   0   0   0   0   0   0] => ['first', 'citizen', '', '', '', '', '', '', '', '']
<class 'numpy.ndarray'>
[138  36 982 144 673 125  16 106   0   0] => ['before', 'we', 'proceed', 'any', 'further', 'hear', 'me', 'speak', '', '']
<class 'numpy.ndarray'>
[34  0  0  0  0  0  0  0  0  0] => ['all', '', '', '', '', '', '', '', '', '']
<class 'numpy.ndarray'>
[106 106   0   0   0   0   0   0   0   0] => ['speak', 'speak', '', '', '', '', '', '', '', '']
<class 'numpy.ndarray'>
[ 89 270   0   0   0   0   0   0   0   0] => ['first', 'citizen', '', '', '', '', '', '', '', '']
<class 'numpy.ndarray'>


In [ ]:
# Generates skip-gram pairs with negative sampling for a list of sequences
# (int-encoded sentences) based on window size, number of negative samples
# and vocabulary size.
def generate_training_data(sequences, window_size, num_ns, vocab_size, seed):
  # Elements of each training example are appended to these lists.
  targets, contexts, labels = [], [], []

  # Build the sampling table for vocab_size tokens.
  sampling_table = tf.keras.preprocessing.sequence.make_sampling_table(vocab_size)

  # Iterate over all sequences (sentences) in dataset.
  for sequence in tqdm.tqdm(sequences):

    # Generate positive skip-gram pairs for a sequence (sentence).
    positive_skip_grams, _ = tf.keras.preprocessing.sequence.skipgrams(
          sequence, 
          vocabulary_size=vocab_size,
          sampling_table=sampling_table,
          window_size=window_size,
          negative_samples=0)

    # Iterate over each positive skip-gram pair to produce training examples 
    # with positive context word and negative samples.
    for target_word, context_word in positive_skip_grams:
      context_class = tf.expand_dims(
          tf.constant([context_word], dtype="int64"), 1)
      negative_sampling_candidates, _, _ = tf.random.log_uniform_candidate_sampler(
          true_classes=context_class,
          num_true=1, 
          num_sampled=num_ns, 
          unique=True, 
          range_max=vocab_size, 
          seed=SEED, 
          name="negative_sampling")

      # Build context and label vectors (for one target word)
      negative_sampling_candidates = tf.expand_dims(
          negative_sampling_candidates, 1)

      context = tf.concat([context_class, negative_sampling_candidates], 0)
      label = tf.constant([1] + [0]*num_ns, dtype="int64")

      # Append each element from the training example to global lists.
      targets.append(target_word)
      contexts.append(context)
      labels.append(label)

  return targets, contexts, labels

In [ ]:
targets, contexts, labels = generate_training_data(
    sequences=sequences, 
    window_size=2, 
    num_ns=4, 
    vocab_size=vocab_size, 
    seed=SEED)
print(len(targets), len(contexts), len(labels))


In [ ]:
BATCH_SIZE = 1024
BUFFER_SIZE = 10000
dataset = tf.data.Dataset.from_tensor_slices(((targets, contexts), labels))
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)
print(dataset)
dataset = dataset.cache().prefetch(buffer_size=AUTOTUNE)
print(dataset)


In [ ]:
class Word2Vec(Model):
  def __init__(self, vocab_size, embedding_dim):
    super(Word2Vec, self).__init__()
    self.target_embedding = Embedding(vocab_size, 
                                      embedding_dim,
                                      input_length=1,
                                      name="w2v_embedding", )
    self.context_embedding = Embedding(vocab_size, 
                                       embedding_dim, 
                                       input_length=num_ns+1)
    self.dots = Dot(axes=(3,2))
    self.flatten = Flatten()

  def call(self, pair):
    target, context = pair
    we = self.target_embedding(target)
    ce = self.context_embedding(context)
    dots = self.dots([ce, we])
    return self.flatten(dots)


In [ ]:
def custom_loss(x_logit, y_true):
      return tf.nn.sigmoid_cross_entropy_with_logits(logits=x_logit, labels=y_true)
embedding_dim = 128

word2vec = Word2Vec(vocab_size, embedding_dim)
word2vec.compile(optimizer='adam',
              loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir="logs")
word2vec.fit(dataset, epochs=20, callbacks=[tensorboard_callback])


In [ ]:
%load_ext tensorboard

fashion_mnist = tf.keras.datasets.fashion_mnist

(x_train, y_train),(x_test, y_test) = fashion_mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

%tensorboard --logdir logs
